In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/creditcardfraud/creditcard.csv


In [2]:
#Credit to:
#https://logitboost.readthedocs.io/examples/Breast_Cancer.html
#https://www.kaggle.com/meryemeerrkik/credit-fraud-recall-98


In [3]:
#Importing libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, scale
from sklearn.model_selection import train_test_split, cross_validate
!pip install logitboost
from logitboost import LogitBoost
from sklearn.metrics import accuracy_score, classification_report

In [4]:
# Reading data
df = pd.read_csv("../input/creditcardfraud/creditcard.csv")

In [5]:

# Defining X & y

df['Amount'] = scale(df['Amount'])

##Handle Imbalanced Data
#1. Find the number of the minority class
number_fraud = len(df[df['Class']==1])
number_non_fraud = len(df[df['Class']==0])
#2. Find the indices of the majority class
index_non_fraud = df[df['Class']==0].index
#.3 Find the indices of the minority class
index_fraud = df[df['Class']==1].index
#4. Randomly sample the majority indices with respect to the number of minority classes
random_indices = np.random.choice(index_non_fraud, number_fraud,replace='False')
len(random_indices)
#5. Concat the minority indices with the indices from step 4
under_sample_indices = np.concatenate([index_fraud,random_indices])
#Get the balanced dataframe - This is the final undersampled data
under_sample_df = df.iloc[under_sample_indices]
under_sample_class_counts = pd.value_counts(under_sample_df['Class'])
# dividing the X and the Y from the dataset 
X = under_sample_df.drop(['Class'], axis = 1) 
y= under_sample_df["Class"]


In [6]:
#Spliting 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [7]:
#fitting the model [LogitBoost]

lboost = LogitBoost(n_estimators= 200, learning_rate = 1)
lboost.fit(X_train, y_train)

LogitBoost(learning_rate=1, n_estimators=200)

In [8]:
#Predicting
y_pred_train = lboost.predict(X_train)
y_pred_test = lboost.predict(X_test)

#Assessing the scores
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)

print('Training accuracy: %.4f' % accuracy_train)
print('Test accuracy:     %.4f' % accuracy_test)

report_train = classification_report(y_train, y_pred_train)
report_test = classification_report(y_test, y_pred_test)
print('Training\n%s' % report_train)
print('Testing\n%s' % report_test)

Training accuracy: 0.9936
Test accuracy:     0.9289
Training
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       383
           1       1.00      0.99      0.99       404

    accuracy                           0.99       787
   macro avg       0.99      0.99      0.99       787
weighted avg       0.99      0.99      0.99       787

Testing
              precision    recall  f1-score   support

           0       0.94      0.93      0.94       109
           1       0.91      0.93      0.92        88

    accuracy                           0.93       197
   macro avg       0.93      0.93      0.93       197
weighted avg       0.93      0.93      0.93       197

